# We first import the relevent env and policy handler for the

In [1]:
!git clone https://github.com/ntasfi/PyGame-Learning-Environment
!pip install gym_ple
!pip install -e ./PyGame-Learning-Environment
!pip install pyvirtualdisplay
!pip install pygame

fatal: destination path 'PyGame-Learning-Environment' already exists and is not an empty directory.


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///home/sagie/Downloads/midTern_RL/midTern_RL/PyGame-Learning-Environment
  Preparing metadata (setup.py) ... done
  Attempting uninstall: ple
    Found existing installation: ple 0.0.1
    Uninstalling ple-0.0.1:
      Successfully uninstalled ple-0.0.1
  Running setup.py develop for ple
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import sys
sys.path.append('/content/PyGame-Learning-Environment')
from ple.games.flappybird import FlappyBird
from ple import PLE
from gym import spaces
import IPython
import numpy as np
import gym
import os
import pyvirtualdisplay
import base64
import warnings
import imageio
warnings.filterwarnings("ignore")


os.environ["SDL_VIDEODRIVER"] = "dummy"

OBSERVATION_MAP = {0: 'player_y',
                   1: "player_vel",
                   2: "next_pipe_dist_to_player",
                   3: "next_pipe_top_y",
                   4: "next_pipe_bottom_y",
                   5: "next_next_pipe_dist_to_player",
                   6: "next_next_pipe_top_y",
                   7: "next_next_pipe_bottom_y",
}

class Game(gym.Env):
    def __init__(self, display_screen=False,
                 force_fps=True,
                 custom_obs=False,
                 pipe_gap=80,
                 custome_observation_map={},
                 preprocess=lambda x: x,
                 reward_shaping=lambda x, y, z: x):

        os.environ["SDL_VIDEODRIVER"] = "dummy"
        game = FlappyBird(pipe_gap=pipe_gap)  # define and initiate the environment
        self.env = PLE(game, fps=30, display_screen=display_screen,
                       force_fps=force_fps)
        self.env.init()
        # list of actions in the environment
        self.actions = self.env.getActionSet()
        # length of actions
        self.action_space = spaces.Discrete(len(self.actions))
        self.custom_obs = custom_obs
        self._observation_map = custome_observation_map if custom_obs else OBSERVATION_MAP
        self.preprocess = preprocess
        self.reward_shaping = reward_shaping
        self.score = 0

    @property
    def observation_space(self):
        return spaces.Box(low=0, high=512, shape=(len(self._observation_map),), dtype=int)

    def _get_rgb(self):
        return self.env.getScreenRGB().transpose(1, 0, 2)

    @property
    def observation_map(self):
        return self._observation_map

    def step(self, action):
        """Take the action chosen and update the reward"""
        reward = self.env.act(self.actions[action])
        if reward < 0:
          reward = -1

        if reward > 0:
          self.score += 1
        state = self.env.getGameState()
        terminal = self.env.game_over()
        reward = self.reward_shaping(reward, terminal, env)
        info = {'score':self.score}
        return self.preprocess(np.array(list(state.values())).astype(int)), reward, terminal, info

    def getGameState(self):
        '''
        PLEenv return gamestate as a dictionary. Returns a modified form
        of the gamestate only with the required information to define the state
        '''
        state = self.env.getGameState()
        h_dist = state['next_pipe_dist_to_player']
        v_dist = state['next_pipe_bottom_y'] - state['player_y']
        vel = state['player_vel']

        return ' '.join([str(vel), str(h_dist), str(v_dist)])

    def reset(self):
        """Resets the game to start a new game"""
        self.env.reset_game()
        state = self.env.getGameState()
        self.score = 0
        return self.preprocess(np.array(list(state.values())).astype(int))

    def render(self, mode='human'):
        """Render the game"""
        return self._get_rgb()

    def seed(self, seed):
        rng = np.random.RandomState(seed)
        self.env.rng = rng
        self.env.game.rng = self.env.rng

        self.env.init()

custome_observation_map = {0: "player_y",
                           1: "player_val",
                           2: "next_pipe_dist_to_player",
                           3: "next_pipe_top_y"} # assing each action with a description of the feature meaning
def preprocess(obs):
  """
  insert your preprocessing code here
  """

  obs = np.array([obs[0], obs[1], obs[2], obs[3]])
  return obs

env = Game(custom_obs=True, preprocess=preprocess, custome_observation_map=custome_observation_map)


pygame 2.6.1 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html
couldn't import doomish
Couldn't import doom


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile


In [3]:
def embed_mp4(filename):
  """Embeds an mp4 file in the notebook."""
  video = open(filename,'rb').read()
  b64 = base64.b64encode(video)
  tag = '''
  <video width="640" height="480" controls>
    <source src="data:video/mp4;base64,{0}" type="video/mp4">
  Your browser does not support the video tag.
  </video>'''.format(b64.decode())

  return IPython.display.HTML(tag)
display = pyvirtualdisplay.Display(visible=0, size=(1400, 900)).start()

In [4]:
from abc import ABC, abstractmethod

class OnPolicyAgent(ABC):
    def __init__(self, action_space, observation_space, gamma, learning_rate):
        """
        Initializes the on-policy RL agent.
        You can add more parameters
        Parameters:
        - action_space: The action space of the environment
        - observation_space: The observation space of the environment
        - gamma: Discount factor for future rewards
        - learning_rate: Learning rate for policy updates
        """
        self.action_space = action_space
        self.observation_space = observation_space
        self.gamma = gamma
        self.learning_rate = learning_rate
        self.policy = None  # Placeholder for policy (to be implemented in subclasses)

    @abstractmethod
    def select_action(self, observation, deterministic=False):
        """
        Select an action based on the current policy.

        Parameters:
        - observation: Current state observation from the environment
        -deterministic: Flag indicating whether to use a deterministic policy (default is False)
        Returns:
        - action: Chosen action based on the policy
        """


    @abstractmethod
    def update_policy(self, transition):
        """
        Update the policy based on the current transition.

        Parameters:
        - transition: Data collected from interactions with the environment (tuple of zero or more values such as acion, reward etc...)
        trasition - ([s_i], [a], [s_j], [a_tag], [reward])
        """

    @abstractmethod
    def train(self, env, num_episodes, log_every):
        """
        Train the agent over a specified number of episodes.

        Parameters:
        - env: The environment to train in
        - num_episodes: Number of training episodes
        """


    @abstractmethod
    def save_policy(self, file_path):
        """
        Save the current policy to a file.

        Parameters:
        - file_path: Path to the file where the policy should be saved
        """
        pass

    @abstractmethod
    def load_policy(self, file_path):
        """
        Load a policy from a file.

        Parameters:
        - file_path: Path to the file from which the policy should be loaded
        """
        pass

    @abstractmethod
    def run_policy(self, env):
        """
        Run the loaded policy in the environment for one episode.

        Parameters:
        - env: The environment in which to run the policy

        Returns:
        - total_reward: Total reward accumulated in the episode
        """
        pass

class MySarsa(OnPolicyAgent):
    def __init__(self, action_space, observation_space, gamma, learning_rate, epsilon=0.1):
        super().__init__(action_space, observation_space, gamma, learning_rate)
        self.policy = np.zeros((*[32] * observation_space.shape[0], action_space.n))
        self.epsilon = epsilon  # Exploration rate

    def select_action(self, observation, deterministic=False):
        """
          Select an action based on the epsilon-greedy strategy.

          Args:
          observation (tuple): The current state or observation.
          deterministic (bool): If True, always select the greedy action (ignore epsilon).

          Returns:
          int: Selected action.
          """

        state = tuple(observation // 16)  # Convert observation to a usable state index if needed
        q_values = self.policy[state]  # Access Q-values for the given state

        if deterministic or np.random.rand() > self.epsilon:
            # Exploit: Choose action with highest Q-value
            action = np.argmax(q_values)
        else:
            # Explore: Choose a random action
            action = np.random.choice(len(q_values))

        return action

    def update_policy(self, transition):
        """
        Update the policy based on the current transition using the SARSA update rule.

        Parameters:
            transition (tuple): A tuple containing:
                - s_i (array-like): Current state.
                - a (int): Current action.
                - s_j (array-like): Next state.
                - a_tag (int): Next action.
                - reward (float): Reward for the transition.

        Updates:
            self.policy: The Q-value for the state-action pair (s_i, a) is updated.
        """
        # Unpack the transition
        s_i, a, s_j, a_tag, reward = transition

        # Convert states to tuples for indexing
        s_i = tuple(s_i // 16)
        s_j = tuple(s_j // 16)

        # SARSA update
        self.policy[s_i][a] += self.learning_rate * (
            reward + self.gamma * self.policy[s_j][a_tag] - self.policy[s_i][a]
        )
    def train(self, env, num_episodes, log_every):
        """
        Train the agent using SARSA.

        Args:
            env (gym.Env): The environment to train the agent in.
            num_episodes (int): Number of episodes to train for.
            log_every (int): Frequency of logging episode statistics.
        """
        max_steps = 2000
        episode_rewards = []
        episode_scores = []
        success_count = []  # To track successful episodes (reaching 10 pipes)

        for episode in range(num_episodes):
            obs = env.reset()
            done = False
            total_reward = 0
            action = self.select_action(obs)
            pipe_count = 0  # Count of pipes cleared in this episode
            info = {}
            for step in range(max_steps):
                prev_obs = obs
                obs, reward, done, info = env.step(action)

                total_reward += reward

                # Count pipes cleared (assumes 'score' tracks pipes passed)
                pipe_count = info.get('score', pipe_count)

                # Select the next action
                a_tag = self.select_action(obs)

                # Create transition
                transition = (prev_obs, action, obs, a_tag, reward)
                self.update_policy(transition)

                # Update the action
                action = a_tag

                if done:
                    break

            score = info.get('score', 0)
            episode_scores.append(score)

            # Determine if the episode is successful
            success = 1 if pipe_count >= 10 else 0
            success_count.append(success)

            episode_rewards.append(total_reward)

            # Logging
            if episode % log_every == 0 or episode == num_episodes - 1:
                print(f"Episode {episode + 1}/{num_episodes}: "
                      f"Total Reward = {total_reward:.2f}, "
                      f"Steps = {step}, Score = {score}, Success = {success}")

            if episode % (num_episodes // 10) == 0: #every 10% of the loop
                directory_path = f'content/{self.__class__.__name__}_ep={self.epsilon}_ga={self.gamma}_lr={self.learning_rate}'
                os.makedirs(directory_path, exist_ok=True)  # Create the directory if it doesn't exist

                self.run_policy(env,embed=True,fname=f'{directory_path}/{episode}.mp4')

        return episode_rewards, success_count, episode_scores

    def save_policy(self, file_path):
        """
        Save the current policy to a file.

        Parameters:
            file_path (str): Path to the file where the policy should be saved.
        """
        np.save(file_path, self.policy)
        print(f"Policy saved to {file_path}")

    def load_policy(self, file_path):
        """
        Load a policy from a file.

        Parameters:
            file_path (str): Path to the file from which the policy should be loaded.
        """
        if not os.path.exists(file_path):
            raise FileNotFoundError(f"Policy file not found: {file_path}")
        self.policy = np.load(file_path)

    def run_policy(self, env,embed=False,fname="vid.mp4"):
        """
        Run the loaded policy in the environment for one episode.

        Parameters:
            env: The environment in which to run the policy.

        Returns:
            float: Total reward accumulated in the episode.
        """
        video = {}
        if embed:
           video = imageio.get_writer(fname, fps=24)

        obs = env.reset()
        done = False
        score = 0
        max_steps = 2000

        for _ in range(max_steps):
            action = self.select_action(obs, deterministic=True)
            obs, reward, done, info = env.step(action)
            score = info.get('score', 0)

            if embed:
              re = env.render()
              video.append_data(re)
            if done:
                break
        if embed:
          embed_mp4(fname)
          video.close()
        return score


class MyQLearning(MySarsa):
    def __init__(self, action_space, observation_space, gamma, learning_rate, epsilon=0.1):
        """
        Initialize the Q-Learning agent.

        Args:
            action_space (int): Number of possible actions.
            observation_space (tuple): Shape of the state space.
            gamma (float): Discount factor for future rewards.
            learning_rate (float): Learning rate for Q-value updates.
            epsilon (float): Exploration rate for epsilon-greedy policy.
        """
        super().__init__(action_space, observation_space, gamma, learning_rate)

    def update_policy(self, transition):
        """
        Update the policy based on the current transition using the Q-Learning update rule.

        Parameters:
            transition (tuple): A single tuple containing:
                - s_i (array-like): Current state.
                - a (int): Current action.
                - s_j (array-like): Next state.
                - reward (float): Reward for the transition.
        """
        # Unpack the transition
        s_i, a, s_j,a_tag, reward = transition

        # Convert states to tuples for indexing
        s_i = tuple(s_i // 16)
        s_j = tuple(s_j // 16)

        # Q-Learning update rule
        max_q_next = np.max(self.policy[s_j])  # Maximum Q-value for the next state
        self.policy[s_i][a] += self.learning_rate * (
            reward + self.gamma * max_q_next - self.policy[s_i][a]
        )

In [ ]:
import pickle
# Define paths
model_file_path = 'content/big_gap_Qlearning.pkl'  # Path to the policy model file

# Load the policy model
try:
    with open(model_file_path, 'rb') as f:
        model = pickle.load(f)
except FileNotFoundError:
    print(f"Model file not found at {model_file_path}. Ensure the model is uploaded to  the correct path")
    raise


# Define the video filename
video_filename = 'simulation_video.mp4'

In [20]:
# Run the model for 10 episodes and evaluate
num_episodes = 10
total_score = []

for episode in range(num_episodes):
    obs = env.reset()
    done = False
    episode_score = 0
    while not done:
        action = model.select_action(obs, deterministic=True)
        obs, reward, done, info = env.step(action)
        episode_score = info['score']

    total_score.append(episode_score)
    print(f"Episode {episode + 1}: Score = {episode_score}")

# Display evaluation results
average_reward = sum(total_score) / num_episodes
print(f"Average Score over {num_episodes} episodes: {average_reward}")

# Record and display a video of one episode
with imageio.get_writer(video_filename, fps=24) as video:
    obs = env.reset()
    done = False
    while not done:
        action = model.select_action(obs, deterministic=True)
        obs, _, done, _ = env.step(action)
        frame = env.render(mode='rgb_array')
        video.append_data(frame)

# Embed video in the notebook
embed_mp4(video_filename)

Episode 1: Score = 10
Episode 2: Score = 13
Episode 3: Score = 19
Episode 4: Score = 23
Episode 5: Score = 1
Episode 6: Score = 16
Episode 7: Score = 64
Episode 8: Score = 25
Episode 9: Score = 48
Episode 10: Score = 12
Average Score over 10 episodes: 23.1
